In [ ]:
%pip install googletrans==4.0.0-rc1 PySocks

Para poder reimportar y que los cambios surjan efecto sin necesidad de reiniciar el kernel, se debe ejecutar la siguiente celda:

In [8]:
import importlib
import tunel_ssh_socks

importlib.reload(tunel_ssh_socks)

<module 'tunel_ssh_socks' from '/mnt/d/DeveloperIA/video-translator/tunel_ssh_socks.py'>

<H1> TODO: Que se puedan generar muchos túneles y muchos proxis, para eso el modulo tunel podría tener arrays de tuneles y otro de proxys </H1>

# Probando librería tunel inverso

In [3]:
# import socket
import tunel_ssh_socks as tunel


# original_socket = socket.socket

# Configuración del servidor SSH y credenciales
ssh_host = "74.208.163.90"  # IP del servidor SSH
local_port = 5559               # Puerto SSH
username = "root"     
path_to_private_key = "./.ssh/tunel"  # Ruta clave privada SSH

tunel.crear_tunel_ssh(local_port, username, ssh_host, path_to_private_key)

tunel.crear_proxy_socks(local_port) 


[PM2] Starting /usr/bin/bash in fork_mode (1 instance)
[PM2] Done.
┌────┬──────────────────┬─────────────┬─────────┬─────────┬──────────┬────────┬──────┬───────────┬──────────┬──────────┬──────────┬──────────┐
│ id │ name             │ namespace   │ version │ mode    │ pid      │ uptime │ ↺    │ status    │ cpu      │ mem      │ user     │ watching │
├────┼──────────────────┼─────────────┼─────────┼─────────┼──────────┼────────┼──────┼───────────┼──────────┼──────────┼──────────┼──────────┤
│ 0  │ neo4j console    │ default     │ N/A     │ fork    │ 27838    │ 4D     │ 0    │ online    │ 0%       │ 7.5mb    │ javier   │ disabled │
│ 29 │ tunel-ssh        │ default     │ N/A     │ fork    │ 26586    │ 0s     │ 0    │ online    │ 0%       │ 1.5mb    │ javier   │ disabled │
└────┴──────────────────┴─────────────┴─────────┴─────────┴──────────┴────────┴──────┴───────────┴──────────┴──────────┴──────────┴──────────┘
Configurando el proxy SOCKS en localhost:5559...


In [6]:
import tunel_ssh_socks as tunel

tunel.obtener_informacion_ip()


'Dirección IP: 83.138.42.107\nPaís: ES\nRegión: Valencia\nCiudad: Alicante'

In [5]:
from googletrans import Translator
import random
import time


def traducir_eng_spa(texto):
    traductor = Translator()
    return traductor.translate(texto, src='en', dest='es').text

# Lista de textos a traducir
textos = ["Hello, world!", "How are you?", "This is a test."]

# Iterar sobre los textos y traducir cada uno
for texto in textos:
    texto_traducido = traducir_eng_spa(texto)
    print(texto_traducido)

    # Genera una pausa aleatoria entre 1 y 2 segundos
    time.sleep(random.uniform(1, 2))


tunel.cerrar_tunel_ssh()


¡Hola Mundo!
¿Cómo estás?
Esto es una prueba.
[PM2] Applying action deleteProcessId on app [tunel-ssh](ids: [ 29 ])
[PM2] [tunel-ssh](29) ✓
┌────┬──────────────────┬─────────────┬─────────┬─────────┬──────────┬────────┬──────┬───────────┬──────────┬──────────┬──────────┬──────────┐
│ id │ name             │ namespace   │ version │ mode    │ pid      │ uptime │ ↺    │ status    │ cpu      │ mem      │ user     │ watching │
├────┼──────────────────┼─────────────┼─────────┼─────────┼──────────┼────────┼──────┼───────────┼──────────┼──────────┼──────────┼──────────┤
│ 0  │ neo4j console    │ default     │ N/A     │ fork    │ 27838    │ 4D     │ 0    │ online    │ 0%       │ 7.5mb    │ javier   │ disabled │
└────┴──────────────────┴─────────────┴─────────┴─────────┴──────────┴────────┴──────┴───────────┴──────────┴──────────┴──────────┴──────────┘


## probando librería tunel inverso (visualización de datos)

In [2]:
# import socket
# original_socket = socket.socket

In [3]:
print(id(original_socket))
# print(dir(instancia))
# print(vars(original_socket))
# atributos = [a for a in dir(original_socket) if not a.startswith('__')]
# print(atributos)
import socket


# Atributos de interés
print("Tipo de Socket:", original_socket.type)
print("Familia del Socket:", original_socket.family)

# Después de conectar o enlazar el socket
# s.bind(("localhost", 0))
# s.listen()
# print("Dirección Local:", s.getsockname())

# Si el socket está conectado a un servidor remoto
# s.connect(("www.ejemplo.com", 80))
# print("Dirección Remota:", s.getpeername())

# Obtener opciones del socket
# timeout = s.getsockopt(socket.SOL_SOCKET, socket.SO_RCVTIMEO)
# print("Timeout de Recepción:", timeout)


15687680
Tipo de Socket: <property object at 0x7f444d0d5120>
Familia del Socket: <property object at 0x7f444d234f40>


In [4]:
# imprimir id actual socket
print(id(socket.socket))

15687680


In [65]:
!pm2 list

┌────┬────────────────────┬──────────┬──────┬───────────┬──────────┬──────────┐
│ id │ name               │ mode     │ ↺    │ status    │ cpu      │ memory   │
├────┼────────────────────┼──────────┼──────┼───────────┼──────────┼──────────┤
│ 0  │ neo4j console      │ fork     │ 0    │ online    │ 0%       │ 7.5mb    │
│ 25 │ tunel-ssh          │ fork     │ 0    │ online    │ 0%       │ 8.0mb    │
│ 26 │ tunel-ssh          │ fork     │ 0    │ online    │ 0%       │ 7.9mb    │
└────┴────────────────────┴──────────┴──────┴───────────┴──────────┴──────────┘


In [8]:
!pm2 delete tunel-ssh

[PM2] Applying action deleteProcessId on app [tunel-ssh](ids: [ 28 ])
[PM2] [tunel-ssh](28) ✓
┌────┬────────────────────┬──────────┬──────┬───────────┬──────────┬──────────┐
│ id │ name               │ mode     │ ↺    │ status    │ cpu      │ memory   │
├────┼────────────────────┼──────────┼──────┼───────────┼──────────┼──────────┤
│ 0  │ neo4j console      │ fork     │ 0    │ online    │ 0%       │ 7.5mb    │
└────┴────────────────────┴──────────┴──────┴───────────┴──────────┴──────────┘


In [ ]:
# import socket

# Restaurar el comportamiento original del socket
# socket.socket = original_socket

In [5]:
import tunel_ssh_socks as tunel

tunel.obtener_informacion_ip()

'Dirección IP: 83.138.42.107\nPaís: ES\nRegión: Valencia\nCiudad: Alicante'

In [ ]:
import tunel_ssh_socks as tunel

tunel.cerrar_tunel_ssh()

[PM2] Applying action deleteProcessId on app [tunel-ssh](ids: [ 21 ])
[PM2] [tunel-ssh](21) ✓
┌────┬──────────────────┬─────────────┬─────────┬─────────┬──────────┬────────┬──────┬───────────┬──────────┬──────────┬──────────┬──────────┐
│ id │ name             │ namespace   │ version │ mode    │ pid      │ uptime │ ↺    │ status    │ cpu      │ mem      │ user     │ watching │
├────┼──────────────────┼─────────────┼─────────┼─────────┼──────────┼────────┼──────┼───────────┼──────────┼──────────┼──────────┼──────────┤
│ 0  │ neo4j console    │ default     │ N/A     │ fork    │ 27838    │ 3D     │ 0    │ online    │ 0%       │ 7.5mb    │ javier   │ disabled │
└────┴──────────────────┴─────────────┴─────────┴─────────┴──────────┴────────┴──────┴───────────┴──────────┴──────────┴──────────┴──────────┘


He ejecutado esto tanto en el servidor remoto con SOCKs como en la máquin WSL local.

# Googletrans API google translator gratis (usar con cuidado) (usar con proxy socks)

In [ ]:
%pip install googletrans==3.1.0a0

## Sin proxy SOCKs


In [5]:
import time
import random
from googletrans import Translator


def traducir_eng_spa(texto):
    traductor = Translator()
    return traductor.translate(texto, src='en', dest='es').text

# Lista de textos a traducir
textos = ["Hello, world!", "How are you?", "This is a test."]

# Iterar sobre los textos y traducir cada uno
for texto in textos:
    texto_traducido = traducir_eng_spa(texto)
    print(texto_traducido)

    # Genera una pausa aleatoria entre 1 y 2 segundos
    time.sleep(random.uniform(1, 2))


¡Hola Mundo!
¿Cómo estás?
Esto es una prueba.


## Usar con proxy socks

In [14]:
puertosocks = 5559

import socket

# Guardar la referencia original del socket
original_socket = socket.socket


## Creamos un proceso robusto en segundo plano que se encarga de mantener el túnel ssh abierto

In [17]:
import subprocess

host_remoto = "74.208.163.90"
usuario = "root"
clave_ssh = ".ssh/tunel"
puerto_local = 5559
nombre_proceso = "tunel-ssh"

cmd = f"pm2 start 'ssh -i {clave_ssh} -N {usuario}@{host_remoto} -D {puerto_local}' --name {nombre_proceso}"
subprocess.run(cmd, shell=True)


[PM2] Starting /usr/bin/bash in fork_mode (1 instance)
[PM2] Done.
┌────┬──────────────────┬─────────────┬─────────┬─────────┬──────────┬────────┬──────┬───────────┬──────────┬──────────┬──────────┬──────────┐
│ id │ name             │ namespace   │ version │ mode    │ pid      │ uptime │ ↺    │ status    │ cpu      │ mem      │ user     │ watching │
├────┼──────────────────┼─────────────┼─────────┼─────────┼──────────┼────────┼──────┼───────────┼──────────┼──────────┼──────────┼──────────┤
│ 0  │ neo4j console    │ default     │ N/A     │ fork    │ 27838    │ 3D     │ 0    │ online    │ 0%       │ 7.5mb    │ javier   │ disabled │
│ 20 │ tunel-ssh        │ default     │ N/A     │ fork    │ 25036    │ 0s     │ 0    │ online    │ 0%       │ 5.1mb    │ javier   │ disabled │
└────┴──────────────────┴─────────────┴─────────┴─────────┴──────────┴────────┴──────┴───────────┴──────────┴──────────┴──────────┴──────────┘


CompletedProcess(args="pm2 start 'ssh -i .ssh/tunel -N root@74.208.163.90 -D 5559' --name tunel-ssh", returncode=0)

No funciona por culpa de que la contraseña está encriptada en el fichero de configuración de proxy socks.

In [17]:
!pm2 list

┌────┬────────────────────┬──────────┬──────┬───────────┬──────────┬──────────┐
│ id │ name               │ mode     │ ↺    │ status    │ cpu      │ memory   │
├────┼────────────────────┼──────────┼──────┼───────────┼──────────┼──────────┤
│ 0  │ neo4j console      │ fork     │ 0    │ online    │ 0%       │ 7.5mb    │
│ 16 │ ssh_tunnel         │ fork     │ 0    │ online    │ 0%       │ 7.9mb    │
└────┴────────────────────┴──────────┴──────┴───────────┴──────────┴──────────┘


In [18]:
# si queremos borrar el proceso
!pm2 delete tunel-ssh

[PM2] Applying action deleteProcessId on app [16](ids: [ '16' ])
[PM2] [ssh_tunnel](16) ✓
┌────┬────────────────────┬──────────┬──────┬───────────┬──────────┬──────────┐
│ id │ name               │ mode     │ ↺    │ status    │ cpu      │ memory   │
├────┼────────────────────┼──────────┼──────┼───────────┼──────────┼──────────┤
│ 0  │ neo4j console      │ fork     │ 0    │ online    │ 0%       │ 7.5mb    │
└────┴────────────────────┴──────────┴──────┴───────────┴──────────┴──────────┘


In [39]:
!lsof -i :{puertosocks}


In [76]:
# !kill -9 19841

In [95]:
!ps aux | grep 'ssh'

root       240  0.0  0.0  15428  1656 ?        Ss   Nov13   0:00 sshd: /usr/sbin/sshd [listener] 0 of 10-100 startups
javier   14789  0.0  0.0      0     0 ?        Z    16:34   0:00 [sshpass] <defunct>
root     24996  0.0  0.0  16880  4760 ?        Ss   09:45   0:00 sshd: javier [priv]
javier   25022  0.0  0.0  17736  4228 ?        S    09:45   0:01 sshd: javier@notty
javier   26478  0.0  0.0   4780  3324 ?        Ss   17:29   0:00 bash /mnt/d/DeveloperIA/video-translator/ssh_tunnel.sh 5559
javier   26479  0.0  0.0  11716  8192 ?        S    17:29   0:00 ssh -i .ssh/tunel -N root@74.208.163.90 -D 5559
javier   26594  0.0  0.0   4780  3296 pts/0    Ss+  17:29   0:00 /bin/bash -c ps aux | grep 'ssh'
javier   26596  0.0  0.0   4024  2000 pts/0    S+   17:29   0:00 grep ssh
root     28723  0.0  0.0  16884  4688 ?        Ss   10:00   0:00 sshd: javier [priv]
javier   28749  0.0  0.0  18480  5592 ?        S    10:00   0:03 sshd: javier@notty


In [ ]:
import requests

def obtener_mi_ip():
    try:
        respuesta = requests.get('https://httpbin.org/ip')
        ip = respuesta.json()['origin']
        return ip
    except Exception as e:
        return f"Error al obtener la IP: {e}"

# Ejecutar la función y mostrar la IP
mi_ip = obtener_mi_ip()
print(f"Mi dirección IP actual es: {mi_ip}")



def obtener_informacion_ip():
    try:
        # Obtener la dirección IP y la información de ubicación
        respuesta_ip = requests.get('https://httpbin.org/ip')
        ip = respuesta_ip.json()['origin']

        # Usar ipinfo.io para obtener información detallada de la ubicación
        respuesta_geo = requests.get(f'https://ipinfo.io/{ip}/json')
        datos_geo = respuesta_geo.json()

        pais = datos_geo.get('country', 'No disponible')
        region = datos_geo.get('region', 'No disponible')
        ciudad = datos_geo.get('city', 'No disponible')

        return f"Dirección IP: {ip}\nPaís: {pais}\nRegión: {region}\nCiudad: {ciudad}"
    except Exception as e:
        return f"Error al obtener la información de la IP: {e}"

# Ejecutar la función y mostrar la información
info_ip = obtener_informacion_ip()
print(info_ip)


print("IP:", obtener_mi_ip())
print(obtener_informacion_ip())

## Ejecutamos servicios mediante el proxy socks

In [18]:
import socket
import socks
import requests
import time
import random
from googletrans import Translator


# Configuración del servidor SSH y credenciales
# ssh_host = "74.208.163.90"  # IP del servidor SSH
# ssh_port = 22               # Puerto SSH
# username = "root"     
# path_to_private_key = "./.ssh/tunel"  # Ruta clave privada SSH

try:

    #  # Configura el proxy SOCKS: Configurar el extremo local del tunel SSH
    local_bind_port = 5559
    remote_bind_address = ('localhost', local_bind_port)

    print(f"Configurando el proxy SOCKS en {remote_bind_address[0]}:{remote_bind_address[1]}...")
    # El argumento True al final indica que se resolverán los nombres de host de manera remota.
    socks.set_default_proxy(socks.SOCKS5, "localhost", local_bind_port, True) 
    socket.socket = socks.socksocket
    socket.AF_INET = 2  # IPv4

    print(obtener_informacion_ip())
    
    def obtener_clima_ny():
        URL = "http://wttr.in/New%20York?format=3"
        print("Realizando solicitud HTTP...")
        respuesta = requests.get(URL, timeout=10)
        return respuesta.text

    # Ejecutar y mostrar la información del clima
    try:
        clima = obtener_clima_ny()
        print(clima)
    except Exception as e:
        print(f"Error al realizar la solicitud HTTP: {e}")

        

    def traducir_eng_spa(texto):
        traductor = Translator()
        return traductor.translate(texto, src='en', dest='es').text

    # Lista de textos a traducir
    textos = ["Hello, world!", "How are you?", "This is a test."]

    # Iterar sobre los textos y traducir cada uno
    for texto in textos:
        try:
            texto_traducido = traducir_eng_spa(texto)
            print(texto_traducido)
        except Exception as e:
            print(f"Error al realizar la traducción: {e}")

        # Pausa
        time.sleep(random.uniform(1, 2))


except Exception as e:
    print(f"Error al establecer la conexión SSH: {e}")

finally:
    # Restaurar el socket original
    print("Restaurando el socket original...")
    socket.socket = original_socket
    


Configurando el proxy SOCKS en localhost:5559...
Dirección IP: 74.208.163.90
País: US
Región: New York
Ciudad: New York City
Realizando solicitud HTTP...
New York: ☀️   +18°C

¡Hola Mundo!
¿Cómo estás?
Esto es una prueba.
Restaurando el socket original...


## Restaurar el proxy socks a su estado original

In [71]:
# Restaurar el comportamiento original del socket
socket.socket = original_socket

# Cosas probaturas

No es necesario hacer esto:
```bash
sudo vi /etc/sysctl.conf
#poner estas lineas al final para desactivar ipv6
net.ipv6.conf.all.disable_ipv6 = 1
net.ipv6.conf.default.disable_ipv6 = 1
net.ipv6.conf.lo.disable_ipv6 = 1


# Para desactivar ipv6 en el kernel
sudo sysctl -p

# Para borrar caches de DNS (depende del sistema, si no se actualiza el DNS)
sudo systemd-resolve --flush-caches

# Para comprobar que se ha desactivado
ip a | grep inet6

# también se puede comprobar con
cat /proc/sys/net/ipv6/conf/all/disable_ipv6
```

Aquí se intentó que el código python creara tanto el tunel inverso como el proxy socks, pero no funcionó:
```python

In [18]:
import paramiko
from paramiko import SSHClient
import socket
import socks
import requests
import time
import random
from googletrans import Translator


# Configuración del servidor SSH y credenciales
ssh_host = "74.208.163.90"  # IP del servidor SSH
ssh_port = 22               # Puerto SSH
username = "root"     # Tu usuario SSH
path_to_private_key = "./.ssh/tunel"  # Ruta a tu clave privada SSH
# private_key_password = "Cony620438292"#getpass.getpass("Ingresa la frase de contraseña de tu clave privada: ")

try:
    # Configura un cliente SSH y conecta al servidor
    ssh = SSHClient()
    ssh.load_system_host_keys()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    my_key = paramiko.RSAKey.from_private_key_file(path_to_private_key)
    print(f"Conectando a SSH a {username}@{ssh_host} -p {ssh_port}...")
    try:
        ssh.connect(hostname=ssh_host, port=ssh_port, username=username, pkey=my_key)
        print("Conexión SSH establecida.")
    except paramiko.ssh_exception.AuthenticationException:
        print("Error: credenciales inválidas.")
        exit()
    except Exception as e:
        print(f"Error al principio de establecer la conexión SSH: {e}")
        exit()    

    # # Configurar el reenvío de puertos
    local_bind_port = puertosocks
    ssh_transport = ssh.get_transport()
    remote_bind_address = ('localhost', local_bind_port)
    # ssh_transport.request_port_forward(address=remote_bind_address[0], port=remote_bind_address[1])

    # Configura el proxy SOCKS
    print(f"Configurando el proxy SOCKS en {remote_bind_address[0]}:{remote_bind_address[1]}...")
    socks.set_default_proxy(socks.SOCKS5, "localhost", local_bind_port, True)
    socket.socket = socks.socksocket
    socket.AF_INET = 2  # IPv4

    def obtener_clima_ny():
        URL = "http://wttr.in/New%20York?format=3"
        print("Realizando solicitud HTTP...")
        respuesta = requests.get(URL, timeout=10)
        return respuesta.text

    # Ejecutar y mostrar la información del clima
    # for _ in range(3):
    try:
        clima = obtener_clima_ny()
        print(clima)
    except Exception as e:
        print(f"Error al realizar la solicitud HTTP: {e}")

        # Pausa
        # time.sleep(random.uniform(1, 2))

    def traducir_eng_spa(texto):
        traductor = Translator()
        return traductor.translate(texto, src='en', dest='es').text

    # Lista de textos a traducir
    textos = ["Hello, world!", "How are you?", "This is a test."]

    # Iterar sobre los textos y traducir cada uno
    for texto in textos:
        try:
            texto_traducido = traducir_eng_spa(texto)
            print(texto_traducido)
        except Exception as e:
            print(f"Error al realizar la traducción: {e}")

        # Pausa
        time.sleep(random.uniform(1, 2))




except Exception as e:
    print(f"Error al establecer la conexión SSH: {e}")

finally:
    # Cerrar la conexión SSH si está abierta
    if ssh:
        ssh.close()
        print("Conexión SSH cerrada.")


Conectando a SSH a root@74.208.163.90 -p 22...
Conexión SSH establecida.
Configurando el proxy SOCKS en localhost:5559...
Realizando solicitud HTTP...
Error al realizar la solicitud HTTP: HTTPConnectionPool(host='wttr.in', port=80): Max retries exceeded with url: /New%20York?format=3 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f242cd0edd0>: Failed to establish a new connection: Error connecting to SOCKS5 proxy localhost:5559: [Errno 111] Connection refused'))
Error al realizar la traducción: PySocks doesn't support IPv6: ('2a00:1450:4003:803::200e', 443, 0, 0)
Error al realizar la traducción: PySocks doesn't support IPv6: ('2a00:1450:4003:803::200e', 443, 0, 0)
Error al realizar la traducción: PySocks doesn't support IPv6: ('2a00:1450:4003:803::200e', 443, 0, 0)
Conexión SSH cerrada.


In [3]:
# import paramiko
# from paramiko import SSHClient
import socket
import socks
import requests
import time
import random
import getpass
from googletrans import Translator


# Configuración del servidor SSH y credenciales
ssh_host = "74.208.163.90"  # IP del servidor SSH
ssh_port = 22               # Puerto SSH
username = "root"     # Tu usuario SSH
path_to_private_key = "/home/javier/.ssh/id_rsa"  # Ruta a tu clave privada SSH
private_key_password = "Cony620438292"#getpass.getpass("Ingresa la frase de contraseña de tu clave privada: ")

try:
    # # Configura un cliente SSH y conecta al servidor
    # ssh = SSHClient()
    # ssh.load_system_host_keys()
    # ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    # my_key = paramiko.RSAKey.from_private_key_file(path_to_private_key, password=private_key_password)
    # print("Conectando a SSH...")
    # ssh.connect(ssh_host, port=ssh_port, username=username, pkey=my_key)
    # print("Conexión SSH establecida.")

    # # Configurar el reenvío de puertos
    # local_bind_port = 5559
    # ssh_transport = ssh.get_transport()
    # remote_bind_address = ('localhost', local_bind_port)
    # ssh_transport.request_port_forward(address=remote_bind_address[0], port=remote_bind_address[1])

    # # Configura el proxy SOCKS
    # socks.set_default_proxy(socks.SOCKS5, "localhost", local_bind_port, True)
    # socket.socket = socks.socksocket
    # socket.AF_INET = 2  # IPv4

    def obtener_clima_ny():
        URL = "http://wttr.in/New%20York?format=3"
        print("Realizando solicitud HTTP...")
        respuesta = requests.get(URL, timeout=10)
        return respuesta.text

    # Ejecutar y mostrar la información del clima
    # for _ in range(3):
    try:
        clima = obtener_clima_ny()
        print(clima)
    except Exception as e:
        print(f"Error al realizar la solicitud HTTP: {e}")

        # Pausa
        # time.sleep(random.uniform(1, 2))

    # def traducir_eng_spa(texto):
    #     traductor = Translator()
    #     return traductor.translate(texto, src='en', dest='es').text

    # # Lista de textos a traducir
    # textos = ["Hello, world!", "How are you?", "This is a test."]

    # # Iterar sobre los textos y traducir cada uno
    # for texto in textos:
    #     try:
    #         texto_traducido = traducir_eng_spa(texto)
    #         print(texto_traducido)
    #     except Exception as e:
    #         print(f"Error al realizar la traducción: {e}")

    #     # Pausa
    #     time.sleep(random.uniform(1, 2))

except Exception as e:
    print(f"Error al establecer la conexión SSH: {e}")

# finally:
    # Cerrar la conexión SSH si está abierta
    # if ssh:
    #     ssh.close()
    #     print("Conexión SSH cerrada.")


Realizando solicitud HTTP...
New York: ☀️   +8°C



In [4]:
from textblob import TextBlob
import time
import random

def traducir_eng_spa(texto):
    # Imprimir el tipo de dato y el contenido de 'texto'
    print(f"Traduciendo: {texto} (tipo: {type(texto)})")

    # Verificar que 'texto' sea una cadena (string)
    if isinstance(texto, str):
        blob = TextBlob(texto)
        return str(blob.translate(to='es'))
    else:
        raise TypeError("El texto a traducir debe ser una cadena (string), no un/a {type(texto)}.")

# Lista de textos a traducir
textos = ["Hello, world!", "How are you?", "This is a test."]

# Iterar sobre los textos y traducir cada uno
for texto in textos:
    try:
        texto_traducido = traducir_eng_spa(texto)
        print(texto_traducido)
    except Exception as e:
        print(f"Error al realizar la traducción: {e}")

    # Pausa
    time.sleep(random.uniform(1, 2))


Traduciendo: Hello, world! (tipo: <class 'str'>)
Error al realizar la traducción: 'list' object has no attribute 'strip'
Traduciendo: How are you? (tipo: <class 'str'>)
Error al realizar la traducción: 'list' object has no attribute 'strip'
Traduciendo: This is a test. (tipo: <class 'str'>)
Error al realizar la traducción: 'list' object has no attribute 'strip'
